In [2]:
##
#   Padroniza o formato dos valores do CNIR
##
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

arquivo = 'd:\\temp\\cnir\\cnirs.csv'
cnirs = pd.read_csv(arquivo, dtype={
                     'cnir.nirf': str,
                     'cnir.codigoIncra': str,
                     'cnir.ni': str,
                     'cnir.municipio': str,
                     'cnir.areaTotal': float
                 })

# Removendo a coluna 'index'
cnirs.drop(cnirs.columns[[0]], axis=1, inplace=True)


In [2]:
# Selecionando registros para criar base de trabalho
cnirs_sem_vinculacao = cnirs[pd.isna(cnirs['cnir.nirf'])]    # sem nirf vinculado
cnirs_sem_vinculacao

,cnir.codigoCnir,cnir.municipio,cnir.areaTotal,cnir.ni,cnir.codigoIncra,cnir.nirf
0,4774565,1100015,543802.6662,003759720024,9500333109803,NaN
1,272599,1100015,514337.0000,20518552934,0000431903229,NaN
2,445583,1100015,40.5059,28108183987,0010400846389,NaN
4,482431,1100015,1879.9579,32458727808,0011710106004,NaN
5,4774566,1100015,6240.5000,003759720024,9500333109994,NaN
...,...,...,...,...,...,...
6141786,4610360,5300108,15.3000,69082189100,9410180009147,NaN
6141787,4613335,5300108,15.3000,69082189100,9410181008209,NaN
6141788,6635792,5300108,15.2900,01038915104,9510304579065,NaN
6141789,4610970,5300108,15.2182,07296169168,9410180305200,NaN


In [3]:
ditr2020 = pd.read_pickle('..//DADOS//ditr2020.pickle')
alvo_cnirs_ditr2020 = cnirs_sem_vinculacao.merge(ditr2020, left_on='cnir.codigoIncra', right_on='ditr2020.codigoIncra', how='inner')
alvo_cnirs_ditr2020.to_pickle('..//DADOS//alvo_cnirs_DITR2020.pickle')
alvo_cnirs_ditr2020


,cnir.codigoCnir,cnir.municipio,cnir.areaTotal,cnir.ni,cnir.codigoIncra,cnir.nirf,ditr2020.nirf,ditr2020.codigoIncra,ditr2020.ni,ditr2020.areaTotal,ditr2020.municipio
0,482431,1100015,1879.9579,32458727808,0011710106004,NaN,29529999,0011710106004,32458727808,1880.7,1101500
1,2793756,1100015,2000.0000,32458727808,6290490163301,NaN,07633076,6290490163301,32458727808,2016.2,1101500
2,461948,1100015,1974.8519,32458727808,0010901252373,NaN,02520214,0010901252373,32458727808,1969.1,1101500
3,461119,1100015,1660.8087,36945170144,0010900010406,NaN,71981500,0010900010406,91479045268,151.2,1100015
4,461119,1100015,1660.8087,36945170144,0010900010406,NaN,62851225,0010900010406,91479045268,148.4,1100015
...,...,...,...,...,...,...,...,...,...,...,...
388585,5841785,5300108,18.2101,04314417668,9501734553770,NaN,80604919,9501734553770,04314417668,18.2,5300108
388586,6494263,5300108,16.9300,22800832134,9999974454106,NaN,89427599,9999974454106,22800832134,16.9,5300108
388587,6782378,5300108,16.7420,09757899615,9510648823305,NaN,92454020,9510648823305,09757899615,16.7,5300108
388588,6624185,5300108,16.0000,21024839168,9510301874619,NaN,10904930,9510301874619,21024839168,16.0,5300108


In [4]:
# Criando colunas igualdade (ig.xxxxx)
alvo_cnirs_ditr2020['ig.ni'] = alvo_cnirs_ditr2020['cnir.ni'] == alvo_cnirs_ditr2020['ditr2020.ni']
alvo_cnirs_ditr2020['ig.codigoIncra'] = alvo_cnirs_ditr2020['cnir.codigoIncra'] == alvo_cnirs_ditr2020['ditr2020.codigoIncra']
alvo_cnirs_ditr2020['ig.municipio'] = alvo_cnirs_ditr2020['cnir.municipio'] == alvo_cnirs_ditr2020['ditr2020.municipio']

# Criando coluna de variação da área entre CNIR e DITR2019
alvo_cnirs_ditr2020['dif'] = np.where(alvo_cnirs_ditr2020['cnir.areaTotal'] > alvo_cnirs_ditr2020['ditr2020.areaTotal'], 
             (alvo_cnirs_ditr2020['cnir.areaTotal'] - alvo_cnirs_ditr2020['ditr2020.areaTotal']) / alvo_cnirs_ditr2020['cnir.areaTotal'], 
             (alvo_cnirs_ditr2020['ditr2020.areaTotal'] - alvo_cnirs_ditr2020['cnir.areaTotal']) / alvo_cnirs_ditr2020['ditr2020.areaTotal'])  
#reduzindo a quantidade  de valores para simplificar o aprendizado
alvo_cnirs_ditr2020['dif'] = alvo_cnirs_ditr2020['dif'].apply(lambda x:int(x * 100))
alvo_cnirs_ditr2020['dif'] = alvo_cnirs_ditr2020['dif'].apply(lambda x: x <= 3)
alvo_cnirs_ditr2020


,cnir.codigoCnir,cnir.municipio,cnir.areaTotal,cnir.ni,cnir.codigoIncra,cnir.nirf,ditr2020.nirf,ditr2020.codigoIncra,ditr2020.ni,ditr2020.areaTotal,ditr2020.municipio,ig.ni,ig.codigoIncra,ig.municipio,dif
0,482431,1100015,1879.9579,32458727808,0011710106004,NaN,29529999,0011710106004,32458727808,1880.7,1101500,True,True,False,True
1,2793756,1100015,2000.0000,32458727808,6290490163301,NaN,07633076,6290490163301,32458727808,2016.2,1101500,True,True,False,True
2,461948,1100015,1974.8519,32458727808,0010901252373,NaN,02520214,0010901252373,32458727808,1969.1,1101500,True,True,False,True
3,461119,1100015,1660.8087,36945170144,0010900010406,NaN,71981500,0010900010406,91479045268,151.2,1100015,False,True,True,False
4,461119,1100015,1660.8087,36945170144,0010900010406,NaN,62851225,0010900010406,91479045268,148.4,1100015,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388585,5841785,5300108,18.2101,04314417668,9501734553770,NaN,80604919,9501734553770,04314417668,18.2,5300108,True,True,True,True
388586,6494263,5300108,16.9300,22800832134,9999974454106,NaN,89427599,9999974454106,22800832134,16.9,5300108,True,True,True,True
388587,6782378,5300108,16.7420,09757899615,9510648823305,NaN,92454020,9510648823305,09757899615,16.7,5300108,True,True,True,True
388588,6624185,5300108,16.0000,21024839168,9510301874619,NaN,10904930,9510301874619,21024839168,16.0,5300108,True,True,True,True


In [5]:
# Eliminação das colunas desnecessárias
del alvo_cnirs_ditr2020['cnir.nirf']
del alvo_cnirs_ditr2020['cnir.municipio']
del alvo_cnirs_ditr2020['cnir.areaTotal']
del alvo_cnirs_ditr2020['cnir.ni']
del alvo_cnirs_ditr2020['cnir.codigoIncra']

del alvo_cnirs_ditr2020['ditr2020.ni']
del alvo_cnirs_ditr2020['ditr2020.areaTotal']
del alvo_cnirs_ditr2020['ditr2020.municipio']

alvo_cnirs_ditr2020.to_pickle('..//DADOS//alvo_cnirs_ditr2020.pickle')

In [3]:
# recuperando a função de aprendizado 
nmArquivo = '..\\DADOS\\incra.pkl'
f = open(nmArquivo, 'rb')
knn = pickle.load(f)    
# r_modelo_c_incra = knn.predict(alvo_cnirs_ditr2020)    

EOFError: Ran out of input

In [16]:
# conjunto de DITRxCNIR classificados (true) pelo modelo
# segundo o modelo, o declarante informou corretamente o codigoIncra na DITR
print(alvo_cnirs_ditr2020[r_modelo_c_incra])

f_modelo_c_incra = np.logical_not(r_modelo_c_incra)
print(alvo_cnirs_ditr2020[f_modelo_c_incra])



NameError: name 'r_modelo_c_incra' is not defined